Desired functions:

* focus on a window if possible (hackish -- come back to [Applescript - Google Chrome activate a certain window - Stack Overflow](https://stackoverflow.com/questions/10366003/applescript-google-chrome-activate-a-certain-window/34375804#34375804))
* focus on tab
* get list of all tabs (in markdown or other format)
* close a tab
* keep an updated list of tabs

In [ ]:
import applescript
from appscript import app, k, its

import numpy as np
import pandas as pd
from pandas import DataFrame, Series

from IPython.display import display, Markdown, Latex, Math

from rdhyee_utils.google_chrome import GoogleChromeTab, GoogleChrome

#import beakerx
#from beakerx import *


In [ ]:
import ipywidgets as widgets
from ipywidgets import Button, Layout

from urllib.parse import urlparse

widgets.Button, widgets.SelectMultiple

In [ ]:
# Chrome version

gc = app('Google Chrome')
gc.version()

In [ ]:
s = "How do I know if spreadsheet cells are merged using google apps script - Stack Overflow 2597"
s[:100]

In [ ]:
gco = GoogleChrome()

for tab in gco.tabs:
    print (tab.title, tab.id, tab.index_in_window)

In [ ]:
class ChromeController():
    def __init__(self):
        self.gc = GoogleChrome()
        self.load_tabs()
        self._create_widgets()

    def load_tabs(self):
        self.df = DataFrame([
    {
        'title': tab.title,
        'id': int(tab.id),
        'window_id': int(tab.window_id),
        'url': tab.url,
        'tab': tab
    }           
    for tab in self.gc.tabs])
        
    def _create_widgets(self):
        self.tabs_list =  widgets.SelectMultiple(
            description='tab_selector',
            layout=Layout(width='50%', height='300px'),
        )

        self.tabs_list.options = self.tabs_options()

        self.close_button = widgets.Button(
            description = "Close"
        )

        self.focus_button = widgets.Button(
            description = "Focus"
        )

        self.output = widgets.Output(
            layout={'border': '1px solid black', 'width':'50%', 'height':'100px'},
        )
        
        self.ui = widgets.VBox([
              widgets.HBox([self.tabs_list,
                                widgets.VBox([self.focus_button, self.close_button])
                    ]),
              self.output
              
        ])

        # tie event handlers
        self.tabs_list.observe(self.tabs_list_change, names='value')            
        self.close_button.on_click(self.close_click)
        self.focus_button.on_click(self.focus_click)


    def tabs_options(self):
        return self.df.apply(lambda row:(row['title'], (row['id'], row['window_id'])), axis=1).to_list()
    
    @property
    def current_tabs(self):
        tabs = []
        for c in self.tabs_list.value:
            tab_id = c[0]
            # window_id = c[1]
            # rawtab = gco.gc.windows.ID(window_id).tabs.ID(tab_id)
            q = f'id == {tab_id}'
            try:
                rawtab = self.df.query(q).iloc[0]['tab']
                tabs.append(rawtab)
            except Exception as e:
                pass
        return tabs

    def tabs_list_change(self, *args):
        with self.output:
            print ("tabs_list_change")
            print(self.tabs_list.value, str(self.current_tabs))
            self.output.clear_output(wait=True)
 
    def close_click(self, *args):
        with self.output:
            print ("close button clicked")
            self.output.clear_output(wait=True)

    def focus_click(self, *args):
        with self.output:
            print ("focus button clicked")
            self.output.clear_output(wait=True)
            # focus on the first selected tab
            tabs = self.current_tabs
            if tabs:
                tabs[0].focus()
            
    def render(self):
        display(self.ui)



In [ ]:
cc = ChromeController()
cc.render()

In [ ]:
df = DataFrame([
    {
        'title': tab.title,
        'id': int(tab.id),
        'window_id': int(tab.window_id),
        'url': tab.url,
        'tab': tab
    }           
    for tab in gco.tabs])

# table = TableDisplay(df)

options = df.apply(lambda row:(row['title'], (row['id'], row['window_id'])), axis=1).to_list()

In [ ]:
tab_selector = widgets.SelectMultiple(
    description='tab_selector',
    layout=Layout(width='50%', height='300px'),
    options = options,
)

output2 = widgets.Output()
display(tab_selector, output2)

def on_value_change(change):
    with output2:
        for c in change['new']:
            # print (c)
        
            tab_id = c[0]
            window_id = c[1]
            # rawtab = gco.gc.windows.ID(window_id).tabs.ID(tab_id)
            q = f'id == {tab_id}'
            try:
                rawtab = df.query(q).iloc[0]['tab']
            except Exception as e:
                print (e)
            else:
                print (tab_id, rawtab)
            # print(change['new'])

tab_selector.observe(on_value_change, names='value')

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.query(q).iloc[0]['tab']

In [ ]:
tab_id = 1865166370
q = f'id == {tab_id}'
rawtab = df.query(q)['tab'][0]
rawtab

In [ ]:
if False:

    import time

    for tab in gco.tabs():
        tab.focus()
        time.sleep(0.5)

In [ ]:
for w in gco.windows:
    print (w.id, w.name, w.index)


In [ ]:
import pyperclip

tabs_md = []
links = []

for w in gco.windows:
    # print (w.index(), w.active_tab_index())
    for (i, tab) in enumerate(w.tabs):
        link_md = '[{}]({})'.format(tab.title, tab.url)
        
        tab_md = {
            'window_id': w.id,
            'tab_id': tab.id,
            'tab_title':tab.title,
            'tab_url': tab.url
        }
        
        links.append(link_md)
        tabs_md.append(tab_md)

links_md = "* " + "\n* ".join(links)        
display(Markdown(links_md))
pyperclip.copy(links_md)

In [ ]:
# gc.windows()[0].active_tab_index.set(9)
# gc.windows()[0].tabs()[6].close()

In [ ]:
# make a DataFrame out the tab data

